In [5]:
# Create a machine to predict the status of customer whether he/she will pursue with the bank

# Import the libraries

import pandas as pd
import numpy as np
import tensorflow as tf

# Import the Dataset

dataset = pd.read_excel('Bank Data.xlsx')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:,-1].values

# Transform male/female label into computable data using label encoder

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
X[:, 2] = encoder.fit_transform(X[:, 2])            # updated array X have gender at 3rd column

# One Hot encode the Geography column

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])], remainder = 'passthrough')
X = np.array(ct.fit_transform(X))

print(X)
print(y)


# Split the data into training and test set

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Always perform Feature scalling before implementing ANN as to avoid misinterpretation of values

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

[[0.0 1.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [0.0 1.0 0.0 ... 1 0 113931.57]
 ...
 [0.0 1.0 0.0 ... 0 1 42085.58]
 [1.0 0.0 0.0 ... 1 0 92888.52]
 [0.0 1.0 0.0 ... 1 0 38190.78]]
[1 0 1 ... 1 1 0]


In [7]:
# Implementing ANN requirements using Tensorflow

ann = tf.keras.models.Sequential()              # create a sequential model from keras class

# Add first layer which serves as input and shallow learning layer, it extracts parameters from the dataset

ann.add(tf.keras.layers.Dense(units= 6, activation= 'relu'))      # number of neurons is 6 and rectifier activation is used for fast learning

# Add second hidden layer for better performance

ann.add(tf.keras.layers.Dense(units= 6, activation= 'relu'))

# Add the final output layer

ann.add(tf.keras.layers.Dense(units = 1, activation= 'sigmoid'))   # using sigmoid activation to get the probabilities of action(i.e. wether to stay or leave the bank), use softmax activation for non binary output

In [8]:
# Compile the ANN

ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'] )   # using binary entropy because output layer provides bianry output, else use categorical_crossentropy for softmax funtion
                                                                                        # adam used for best stochaistic gradient decent
                                                                                        # accuracy will be the metrics on which ann performance will be measured

# TRAINING THE ANN 

ann.fit(X_train, y_train, batch_size= 32, epochs= 100)      # batch size and epochs can vary depending on training set


Epoch 1/100
250/250 [==============================] - 1s 984us/step - loss: 0.6353 - accuracy: 0.6060
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4920 - accuracy: 0.7922
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4211 - accuracy: 0.8156
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4195 - accuracy: 0.8150
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4062 - accuracy: 0.8235
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3913 - accuracy: 0.8338
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3734 - accuracy: 0.8474
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3637 - accuracy: 0.8492
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3635 - accuracy: 0.8478
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3583 - accu

In [9]:
# Predict the results based on test set

y_pred = ann.predict(X_test) # return the values in probabilities
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1))

# Make the confusion matrix and check accuracy score

from sklearn.metrics import accuracy_score , confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]
[[1517   78]
 [ 201  204]]


0.8605